In [2]:
import pandas as pd

# Test ETL of CSV

In [12]:
df = pd.read_csv('F:/Brightwheel/x_ca_omcc_providers.csv', header=None)

In [13]:
df.head(10)

,0,1,2,3,4,5,6
0,La Rue Park Child Development Center,Child Care Center,50 Atrium Way,Davis,CA,95616,5307538716
1,Carousel Preschool,Child Care Center,8328 Airport Blvd.,Los Angeles,CA,90045,3102166641
2,Adventure Club - Quail Glen,Child Care Center,1250 Canevari Dr,Roseville,CA,95747,9167727529
3,Adventure Club - Coyote Ridge,Child Care Center,1751 Morningstar Dr,Roseville,CA,95747,9167727271
4,Azusa Discovery Center,Child Care Center,155 W. Arrow Highway,Azusa,CA,91702,6263341806
5,Light of Christ Children's Center,Child Care Center,336 South Kalmia Street,Escondido,CA,92025,7607456849
6,Little Peoples Corner Preschool/daycare,Child Care Center,3844 Walnut Drive #c,Eureka,CA,95503,7074450339
7,Kindercare Learning Center LLC,Child Care Center,5448 San Juan Ave,Citrus Heights,CA,95610,9169615599
8,Olive Knolls Christian School,Child Care Center,6201 Fruitvale Ave.,Bakersfield,CA,93308,6613933566
9,Peace Lutheran Early Childhood Education Center,Child Care Center,928 San Juan Rd,Sacramento,CA,95834,9169274060


Need to add columns

In [14]:
df.columns = ['Name', 'Type', 'Address', 'City', 'State', 'Zip', 'Phone']

In [15]:
df.head(2)

,Name,Type,Address,City,State,Zip,Phone
0,La Rue Park Child Development Center,Child Care Center,50 Atrium Way,Davis,CA,95616,5307538716
1,Carousel Preschool,Child Care Center,8328 Airport Blvd.,Los Angeles,CA,90045,3102166641


In [48]:
df.dtypes

Name       object
Type       object
Address    object
City       object
State      object
Zip         int64
Phone       int64
dtype: object

Test inserting data into providers

In [17]:
import sqlite3

In [18]:
conn = sqlite3.connect('test.db')

In [19]:
cursor = conn.cursor()

Upsert - allow the address and phone to change for a provider in the same zip - they may have moved

In [20]:
insert_csv = """
INSERT INTO Providers (Name, Type, Address, City, State, Zip, Phone)
VALUES(?, ?, ?, ?, ?, ?, ?)
ON CONFLICT(Name, Zip) DO UPDATE SET
Address = excluded.Address, Phone = excluded.Phone
"""

In [21]:
cursor.execute(insert_csv, ('Name1', 'Child Care Center', 'Add1', 'City1', 'CA', 123, 1234))
conn.commit()

In [25]:
cursor.execute("SELECT * FROM Providers")
results = cursor.fetchall()
for res in results:
    print(res)

('Name1', 'Child Care Center', 'Add1', 'City1', 'CA', 123, 1234, None, None)


In [28]:
cursor.execute(insert_csv, ('Name1', 'Child Care Center', 'Add2', 'City2', 'CA', 123, 4567))
conn.commit()

In [30]:
cursor.execute("SELECT * FROM Providers")
results = cursor.fetchall()
for res in results:
    print(res)

('Name1', 'Child Care Center', 'Add2', 'City1', 'CA', 123, 4567, None, None)


In [31]:
conn.close()

Test adding all data

In [50]:
for idx, row in df.iterrows():
    print(row['Name'].upper(), row['Type'].upper(), row['Address'].upper(), row['City'].upper(), row['State'].upper(), int(row['Zip']), int(row['Phone']))
    break

LA RUE PARK CHILD DEVELOPMENT CENTER CHILD CARE CENTER 50 ATRIUM WAY DAVIS CA 95616 5307538716


In [45]:
conn = sqlite3.connect('test.db')
cursor = conn.cursor()

Normalize data when inserted - make upper case

In [51]:
for idx, row in df.iterrows():
    cursor.execute(insert_csv, (row['Name'].upper(), row['Type'].upper(), row['Address'].upper(), row['City'].upper(), row['State'].upper(), int(row['Zip']), int(row['Phone'])))    

AttributeError: 'float' object has no attribute 'upper'

In [54]:
df['Name'] = df['Name'].astype(str)
df['Type'] = df['Type'].astype(str)
df['Address'] = df['Address'].astype(str)
df['City'] = df['City'].astype(str)
df['State'] = df['State'].astype(str)

In [55]:
for idx, row in df.iterrows():
    cursor.execute(insert_csv, (row['Name'].upper(), row['Type'].upper(), row['Address'].upper(), row['City'].upper(), row['State'].upper(), int(row['Zip']), int(row['Phone'])))    

In [56]:
cursor.execute("SELECT * FROM Providers LIMIT 5")
results = cursor.fetchall()
for res in results:
    print(res)

('LA RUE PARK CHILD DEVELOPMENT CENTER', 'CHILD CARE CENTER', '50 ATRIUM WAY', 'DAVIS', 'CA', 95616, 5307538716, None, None)
('CAROUSEL PRESCHOOL', 'CHILD CARE CENTER', '8328 AIRPORT BLVD.', 'LOS ANGELES', 'CA', 90045, 3102166641, None, None)
('ADVENTURE CLUB - QUAIL GLEN', 'CHILD CARE CENTER', '1250 CANEVARI DR', 'ROSEVILLE', 'CA', 95747, 9167727529, None, None)
('ADVENTURE CLUB - COYOTE RIDGE', 'CHILD CARE CENTER', '1751 MORNINGSTAR DR', 'ROSEVILLE', 'CA', 95747, 9167727271, None, None)
('AZUSA DISCOVERY CENTER', 'CHILD CARE CENTER', '155 W. ARROW HIGHWAY', 'AZUSA', 'CA', 91702, 6263341806, None, None)


In [57]:
conn.close()

# Test reading from internal data source

In [59]:
df2 = pd.read_json('https://bw-interviews.herokuapp.com/data/providers')

In [61]:
df2.head()

,providers
0,"{'id': 'prvdrs_xTVZPYTrIzsRBVJojkuHkRh6', 'pro..."
1,"{'id': 'prvdrs_9XYNilFfFjGeJjWbjP2uvLje', 'pro..."
2,"{'id': 'prvdrs_6fwq7dwGBVctHrZWbpBuZhWJ', 'pro..."
3,"{'id': 'prvdrs_8YF3tyi0eeqlJm1UIeOZVx68', 'pro..."
4,"{'id': 'prvdrs_nH2OzYh77wA1vs2aXsMqfNLx', 'pro..."


In [64]:
import requests

In [65]:
response = requests.get('https://bw-interviews.herokuapp.com/data/providers')

In [66]:
response.status_code

200

In [68]:
json_output = response.json()

In [70]:
for row in json_output['providers']:
    print(row)
    break

{'id': 'prvdrs_xTVZPYTrIzsRBVJojkuHkRh6', 'provider_name': 'La Rue Park Child Development Center', 'phone': '(530) 753-8716', 'email': 'snfaria@ucdavis.edu', 'owner_name': 'Elizabeth Wall'}


In [71]:
import re

In [79]:
insert_internal = """
INSERT INTO InternalSource (ID, Name, Phone, Email, Owner)
VALUES(?, ?, ?, ?, ?);
"""

In [72]:
def phoneNormalize(phone_number):
    return re.sub("[^0-9]", "", phone_number)

In [82]:
def toUpperNormalize(value):
    if value:
        return value.upper()
    return value

In [73]:
phoneNormalize("(530) 753-8716")

'5307538716'

In [74]:
conn = sqlite3.connect('test.db')
cursor = conn.cursor()

In [83]:
for row in json_output['providers']:
    cursor.execute(insert_internal, (toUpperNormalize(row['id']), toUpperNormalize(row['provider_name']), phoneNormalize(row['phone']), toUpperNormalize(row['email']), toUpperNormalize(row['owner_name'])))

In [84]:
conn.commit()

In [85]:
cursor.execute("SELECT * FROM InternalSource LIMIT 5")
results = cursor.fetchall()
for res in results:
    print(res)

('PRVDRS_XTVZPYTRIZSRBVJOJKUHKRH6', 'LA RUE PARK CHILD DEVELOPMENT CENTER', 5307538716, 'SNFARIA@UCDAVIS.EDU', 'ELIZABETH WALL')
('PRVDRS_9XYNILFFFJGEJJWBJP2UVLJE', 'CAROUSEL PRESCHOOL', 3102166641, 'JANE.UY@CAROUSELSCHOOL.COM', 'SHANEKA KITTRELL')
('PRVDRS_6FWQ7DWGBVCTHRZWBPBUZHWJ', 'ADVENTURE CLUB - QUAIL GLEN', 9167727529, 'JRUTLEDGE@ROSEVILLE.CA.US', 'DEWAYNE BUNNING')
('PRVDRS_8YF3TYI0EEQLJM1UIEOZVX68', 'ADVENTURE CLUB - COYOTE RIDGE', 9167727271, 'VSISNEROZ@ROSEVILLE.CA.US', 'ERNEST EDMONDSON')
('PRVDRS_NH2OZYH77WA1VS2AXSMQFNLX', 'AZUSA DISCOVERY CENTER', 6263341806, 'JHDMADDOX60@MSN.COM', 'WILLIAM BLISS')


In [86]:
conn.close()

# Update data from InternalSource to Providers